# Eat Safe, Love

## Notebook Set Up

In [22]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [23]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [24]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [25]:
# review the collections in our database
print("Collections in 'uk_food':", db.list_collection_names())


Collections in 'uk_food': ['establishments']


In [26]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [27]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}
# Use count_documents to display the number of documents in the result
hygiene_20_count = establishments.count_documents(query)
print(f"Number of establishments with a hygiene score of 20: {hygiene_20_count}")
first_hygiene_20 = establishments.find_one(query)
print("First establishment with a hygiene score of 20:")
# Display the first document in the results using pprint
pprint(first_hygiene_20)


Number of establishments with a hygiene score of 20: 41
First establishment with a hygiene score of 20:
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65c5dcba7b710f10b9200ee3'),
 'geocode': {'latitude': '50.769705', 'longitude': '0.27694'},
 'latitude': None,
 'links': [{'href': 'https://ap

In [28]:
# Convert the result to a Pandas DataFrame
df_hygiene_20 = pd.DataFrame(list(establishments.find(query)))

# Display the number of rows in the DataFrame
print("Number of rows in DataFrame:", len(df_hygiene_20))

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df_hygiene_20.head(10))


NameError: name 'pd' is not defined

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName": "London", "RatingValue": {"$gte": 4}}

# Use count_documents to display the number of documents in the result
london_rating_4_count = establishments.count_documents(query)

# Display the first document in the results using pprint
print(f"Number of London establishments with a RatingValue >= 4: {london_rating_4_count}")
first_london_rating_4 = establishments.find_one(query)
print("First London establishment with a RatingValue >= 4:")
pprint(first_london_rating_4)


In [ ]:
# Convert the result to a Pandas DataFrame
df_london_rating_4 = pd.DataFrame(list(establishments.find(query)))

# Display the number of rows in the DataFrame
print("Number of rows in DataFrame:", len(df_london_rating_4))

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df_london_rating_4.head(10))


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.490142
longitude = 0.083840

query = query = {"RatingValue": 5, "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search}, "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search}}

sort = [("scores.Hygiene", 1)]

top_5_hygiene = establishments.find(query).sort(sort).limit(5)

# Print the results
print("Top 5 establishments with a RatingValue of 5, by lowest hygiene score near 'Penang Flavours':")
for doc in top_5_hygiene:
    pprint(doc)


In [ ]:
# Convert result to Pandas DataFrame

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest


pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]
hygiene_0_grouped = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print("Number of establishments in each Local Authority area with a hygiene score of 0:")

# Print the first 10 results
for doc in hygiene_0_grouped[:10]:  # Displaying first 10 for brevity
    pprint(doc)


In [ ]:
# Convert the result to a Pandas DataFrame
df_hygiene_0_grouped = pd.DataFrame(hygiene_0_grouped)

# Display the number of rows in the DataFrame
print("Number of rows in DataFrame:", len(df_hygiene_0_grouped))

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df_hygiene_0_grouped.head(10))
